In [1]:
import dhlab.nbtext as nb
import json

In [2]:
def interesting_urns(df, limit):
    jaccard = dict((df > 0).astype(int).sum(axis=1))
    res = []
    for urn in jaccard:
        if jaccard[urn] >= limit:
            res.append(urn)
    return res

def get_para(para_list):
    import ast
    paras = []
    for p in para_list:
        paras.append(ast.literal_eval(p))
    urns = [x[0] for x in paras]
    paradict = {u:[x[1] for x in paras if x[0] == u] for u in urns}
    paragraphs = dict()
    for u in paradict:
        paragraphs[u] = get_paragraphs(u, paradict[u])
    return paragraphs

def interesting_paragraphs(urns, wordbag, limit):
    interest = dict()
    for u in urns:
        paragraphs = nb.wordbag_eval_para(wordbag, [u])
        for x in paragraphs:
            if len(paragraphs[x]) >= limit:
                #rint(paragraphs[x])
                interest[x] = paragraphs[x]
    return interest

import requests
def get_paragraphs(urn, paras):
    """Return paragraphs for urn"""
    param = dict()
    param['paragraphs'] = paras
    param['urn'] = urn
    r = requests.get("https://api.nb.no/ngram/paragraphs", json=param)
    return r.json()

In [3]:
def major_urns(df):
    res = []
    means = df.mean()
    for urn in df.index:
        good_urn = True
        for theme in means.index:
            good_urn &= df[theme][urn] > means[theme]
        if good_urn:
            res.append(urn)
    return res

In [4]:
#temaene ble lagret med annen notebook - Heidis definisjoner_0_november
with open('heidis_temadefinisjoner_desember.json') as f:
    first = json.load(f)

import json

json.dump(first, open('heidis_temadefinisjoner.json', 'w', encoding='utf-8'))

In [5]:
list_of_themes = list(first.keys())
list_of_themes

['“Det moderlige(1)/ The motherly”',
 '“Det moderlige(2)/ The motherly”',
 '“Det moderlige(3)/ The motherly”',
 '“Det moderlige(4)/ The motherly”',
 '“Barmhjertighed (1)/charity”',
 '“Barmhjertighed (2)/charity”',
 '“Skjønne piker/ beautiful girls”',
 '“Moderlige inntrykk/Motherly impressions”',
 '“Kvinnen i samfunnet/Woman in society”',
 '“Pigelivets begrensninger/the restrictions of a girl’s life”',
 '“Dansens metafor/the metaphor of dance”',
 '“Kvindekjønnets last (1)/The vice of the female sex”',
 '“Kvindekjønnets last (2)/The vice of the female sex”',
 '“Woman’s happiness”',
 '“Kvinnens behov/woman’s needs”',
 '“En glødende sjels livsvilkår/ the life conditions of a fiery soul”',
 '“Kvindens frihed/Woman’s freedom”',
 '“Guds skapelse av mennesket (1)/God’s creation of man”',
 '“Guds skapelse av mennesket (2)/God’s creation of man”',
 '“Guds skapelse av kvinden / God’s creation of woman”',
 '“Kvindens underskastelse/ Woman’s submission”',
 '“Kvindens menneskeverd/ Woman’s dignity”'

Her er URNene det skal søkes i

In [7]:
urns = nb.book_urn(period=(1830, 1880), limit=12000)

len(urns)

6803

In [8]:

round(90*7/100)

6

## Evaluering av bøker


Her er datarammen for klassifikasjon av bøker, med frekvens av hvert tema.

In [9]:
# sett inn et temanavn eller bare bruk indeks inn i temalisten, som her - 0 er første element 1 er andre osv.
books = dict()
for i in range(len(list_of_themes)):
    books[i] = nb.wordbag_eval(first[list_of_themes[i]], urns)

In [10]:
books[0].head(5)

,Familien_,Hjem_,Livet_,Moderlige_,Omfatte_,Qvindes_,Rest_,Sædelighed_,snitt
2010011903029,44.0,0.0,4686.0,284.0,53.0,8.0,24585.0,2.0,3707.750
2016102148034,3.0,42.0,1168.0,200.0,3.0,174.0,18671.0,0.0,2532.625
2013042608062,3.0,48.0,1048.0,207.0,0.0,145.0,18711.0,0.0,2520.250
2015010648113,3.0,36.0,1052.0,196.0,0.0,162.0,18170.0,0.0,2452.375
2009091403021,96.0,35.0,1228.0,294.0,13.0,465.0,17469.0,3.0,2450.375


In [16]:
theme_eval = nb.wordbag_eval(first[list_of_themes[0]], urns)

### Hva er gjennomsnittet for temaene?

Gjennomsnittet tas over alle bøkene.

In [17]:
theme_eval.mean()

Familien_        7.531549
Hjem_           10.065227
Livet_          39.500532
Moderlige_      12.330698
Omfatte_         2.203132
Qvindes_        10.052151
Rest_          683.311388
Sædelighed_      0.687700
snitt           95.710297
dtype: float64

# Se på effekten av `major_urns`  

Den plukker ut de URN-ene som har best sjanse for treff.

In [18]:
sorted_urns = theme_eval.loc[major_urns(theme_eval)]


In [19]:
sorted_urns.head(20).style.background_gradient()

,Familien_,Hjem_,Livet_,Moderlige_,Omfatte_,Qvindes_,Rest_,Sædelighed_,snitt
2009091403021,96,35,1228,294,13,465,17469,3,2450.38
2008100603020,33,16,886,114,18,222,9201,5,1311.88
2008100603016,23,42,801,71,7,35,8366,1,1168.25
2008112403025,11,13,722,28,26,159,7356,3,1039.75
2006112200022,11,13,737,25,29,176,7297,5,1036.62
2006111400039,48,47,736,102,8,184,6653,17,974.375
2006112101047,9,17,498,111,19,165,6591,1,926.375
2010021503046,69,62,311,85,6,117,6252,5,863.375
2015061048039,132,70,355,152,13,310,5743,3,847.25
2013072208224,9,76,717,75,4,89,5428,1,799.875


In [20]:
sorted_urns.mean()

Familien_        35.307692
Hjem_            46.213675
Livet_          242.538462
Moderlige_       68.940171
Omfatte_          9.598291
Qvindes_         78.324786
Rest_          2922.213675
Sædelighed_       6.333333
snitt           426.183761
dtype: float64

# Hovedloop - generer avsnitt

In [21]:
import re
first.keys()
limit = dict()
for k in first.keys():
    m = re.findall("[0-9]", k)
    limit[k] = m[0]

IndexError: list index out of range

In [ ]:
limit

Define subset of first - the list of themes.


In [12]:
first.keys()

dict_keys(['“Det moderlige(1)/ The motherly”', '“Det moderlige(2)/ The motherly”', '“Det moderlige(3)/ The motherly”', '“Det moderlige(4)/ The motherly”', '“Barmhjertighed (1)/charity”', '“Barmhjertighed (2)/charity”', '“Skjønne piker/ beautiful girls”', '“Moderlige inntrykk/Motherly impressions”', '“Kvinnen i samfunnet/Woman in society”', '“Pigelivets begrensninger/the restrictions of a girl’s life”', '“Dansens metafor/the metaphor of dance”', '“Kvindekjønnets last (1)/The vice of the female sex”', '“Kvindekjønnets last (2)/The vice of the female sex”', '“Woman’s happiness”', '“Kvinnens behov/woman’s needs”', '“En glødende sjels livsvilkår/ the life conditions of a fiery soul”', '“Kvindens frihed/Woman’s freedom”', '“Guds skapelse av mennesket (1)/God’s creation of man”', '“Guds skapelse av mennesket (2)/God’s creation of man”', '“Guds skapelse av kvinden / God’s creation of woman”', '“Kvindens underskastelse/ Woman’s submission”', '“Kvindens menneskeverd/ Woman’s dignity”'])

In [16]:
for theme in first:
    theme_eval = nb.wordbag_eval(first[theme], urns)
    sorted_urns = theme_eval.loc[major_urns(theme_eval)]
    print(theme,'antall urner', len(sorted_urns), '\n\n')
    filename = "".join([c for c in theme if c.isalpha() or c.isdigit() or c==' ']).rstrip() + '.json'
    theme_dict = dict()
    books = 0
    for urn in nb.pure_urn(list(sorted_urns.index)):
        books += 1
        try:
            paras = interesting_paragraphs( [urn], first[theme], limit[theme])
            if len(paras) > 0:
                print(', '.join([str(x) for x in nb.metadata(urn)[0]]), len(paras))
            for w in paras:
                theme_dict[w] = paras[w]
        except:
            True
            #print('problemer med:',  ', '.join([str(x) for x in nb.metadata(urn)[0]]))
        if books % 100 == 0:
            print("prosessert", books, 'bøker', "\n")
    print('\n')      
    json.dump(theme_dict, open(filename, 'w', encoding='utf-8'))

“Kvindens Sandhed” (Minst 3 av 4). antall urner 580 


prosessert 100 bøker 

prosessert 200 bøker 

prosessert 300 bøker 

prosessert 400 bøker 

prosessert 500 bøker 



“Kvinde, lesning og religion” (Minst 4 av 5) antall urner 5 




“Forfatterinder og religion”  (Minst 3 av 4) antall urner 78 




“Kvindens posisjon” (Minst 4 av 6) antall urner 42 




“Kvinden og offentlig hygiene” (Minst 4 av 6) antall urner 174 


prosessert 100 bøker 



